In [1]:
from model import preprocess_data_lgbm as preprocess_data
import lightgbm as lgb

import numpy as np
import pandas as pd
import os

from utils import create_train_test_group

from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    accuracy_score,
    roc_auc_score,
)

In [2]:
gbdt_db0_model = "/home/user14/Cyber/MroPred/output/lgbm/benchmark/lgbm_tuning_results_gbdt_db0_mp1_pt1_as_weekly_tw8/lgbm_tuning_experiment/train_lgbm_fe064_00001_1_learning_rate=0.0358,max_depth=19,num_leaves=368_2025-06-09_02-32-54/checkpoint_000000/model.txt"

gbdt_db1_model = "/home/user14/Cyber/MroPred/output/lgbm/our_model/lgbm_tuning_results_gbdt_db1_mp1_pt1_as_weekly_tw8/lgbm_tuning_experiment/train_lgbm_89478_00002_2_learning_rate=0.0099,max_depth=8,num_leaves=959_2025-06-09_10-22-05/checkpoint_000000/model.txt"

rf_db0_model = "/home/user14/Cyber/MroPred/output/lgbm/benchmark/lgbm_tuning_results_rf_db0_mp1_pt1_as_weekly_tw8/lgbm_tuning_experiment/train_lgbm_1a76b_00000_0_bagging_fraction=0.7738,bagging_freq=7,feature_fraction=0.7924,max_depth=18,num_leaves=425_2025-06-09_09-50-21/checkpoint_000000/model.txt"

rf_db1_model = "/home/user14/Cyber/MroPred/output/lgbm/our_model/lgbm_tuning_results_rf_db1_mp1_pt1_as_weekly_tw8/lgbm_tuning_experiment/train_lgbm_88c5d_00000_0_bagging_fraction=0.6658,bagging_freq=4,feature_fraction=0.7009,max_depth=15,num_leaves=577_2025-06-09_02-01-00/checkpoint_000000/model.txt"

dart_db0_model = "/home/user14/Cyber/MroPred/output/lgbm/benchmark/lgbm_tuning_results_dart_db0_mp1_pt1_as_weekly_tw8/lgbm_tuning_experiment/train_lgbm_c0510_00007_7_learning_rate=0.0211,max_depth=17,num_leaves=314_2025-06-09_10-02-09/checkpoint_000000/model.txt"

dart_db1_model = "/home/user14/Cyber/MroPred/output/lgbm/our_model/lgbm_tuning_results_dart_db1_mp1_pt1_as_weekly_tw8/lgbm_tuning_experiment/train_lgbm_2388e_00014_14_learning_rate=0.0588,max_depth=17,num_leaves=144_2025-06-09_02-19-38/checkpoint_000000/model.txt"


In [3]:
# ------------------------------------------
# best_model_path = gbdt_db0_model
# best_model_path = rf_db0_model
# best_model_path = dart_db0_model
# best_model_path = gbdt_db1_model
# best_model_path = rf_db1_model
best_model_path = dart_db1_model

# trial_name = 'GBDT Benchmark 2018'
# trial_name = 'RF Benchmark 2018'
# trial_name = 'DART Benchmark 2018'
# trial_name = 'GBDT Benchmark 2019'
# trial_name = 'RF Benchmark 2019'
# trial_name = 'DART Benchmark 2019'

# trial_name = 'GBDT Our_model 2018'
# trial_name = 'RF Our_model 2018'
# trial_name = 'DART Our_model 2018'
# trial_name = 'GBDT Our_model 2019'
# trial_name = 'RF Our_model 2019'
trial_name = 'DART Our_model 2019'
# ------------------------------------------
# purchase_year = 2018
purchase_year = 2019

# ------------------------------------------
our_model_benchmark = "our_model"
# our_model_benchmark = "benchmark"

# ------------------------------------------
csv_file_name = "./Data/mro_daily_clean.csv"
target_mro = ["mro"]

maintain_repair_mro = "full"

add_mro_prev = True
add_purchase_time = True

agg_fun = ["mean", "sum", "max", "min", "std", "skew"]



# ------------------------------------------
agg_scale='weekly'

# ------------------------------------------
if our_model_benchmark == "benchmark":
    add_driver_behavior = False
else:
    add_driver_behavior = True

data_folder = './Data/different_year_comparison'
file_name = f'data_{purchase_year}_{our_model_benchmark}.gzip'

data_lgbm_path = os.path.join(data_folder, file_name)

data_lgbm_path = os.path.abspath(data_lgbm_path)

In [4]:
lgbm_different_year_compare_folder = './output/lgbm'
os.makedirs(lgbm_different_year_compare_folder, exist_ok=True)
lgbm_different_year_compare_file_name = 'lgbm_different_year_comparison.csv'
lgbm_different_year_compare_file_path = os.path.join(lgbm_different_year_compare_folder, lgbm_different_year_compare_file_name)

In [5]:
if os.path.isfile(data_lgbm_path):
    print(f"Data file {data_lgbm_path} exists.")
    data_lgbm = pd.read_parquet(data_lgbm_path)
else:
    print(f"{data_lgbm_path} does not exist.")
    data = preprocess_data(
        file_name=csv_file_name,
        target_mro=target_mro,
        maintain_repair_mro=maintain_repair_mro,
        add_mro_prev=add_mro_prev,
        add_purchase_time=add_purchase_time,
        add_driver_behavior=add_driver_behavior,
        agg_scale=agg_scale,
        agg_fun=agg_fun,
        time_window = 8,
    )
    original_data = pd.read_csv(csv_file_name, index_col=0, engine="pyarrow")
    if purchase_year is not None and "purchase_yr_nbr" in original_data.columns:
        selected_ids = original_data.loc[
            original_data["purchase_yr_nbr"] == purchase_year, "id"
        ].unique()
        print(
            f"Filtering by purchase_year={purchase_year}, selected {len(selected_ids)} ids."
        )
        data = data[data["id"].isin(selected_ids)]
    else:
        selected_ids = data["id"].unique()
        print(f"No Purchase Year Selected, selected {len(selected_ids)} ids.")

    data_lgbm = create_train_test_group(
        data=data,
        sample_frac=1.0,
        test_size=0.1,
        valid_size=0.1,
        random_state=42,
    )

    data_lgbm.to_parquet(data_lgbm_path, compression="gzip", engine="pyarrow")

Data file /home/user14/Cyber/MroPred/Data/different_year_comparison/data_2019_our_model.gzip exists.


In [6]:

data_path = data_lgbm


In [7]:
booster = lgb.Booster(model_file=best_model_path)
# data_lgbm = pd.read_parquet(data_path)

In [8]:
def prepare_data(data: pd.DataFrame):
    """Load and split the dataset into train, validation, and test sets."""

    train_dataset = data[data["group"] == "train"]
    valid_dataset = data[data["group"] == "valid"]
    test_dataset = data[data["group"] == "test"]

    train_dataset = train_dataset.drop(["group", "id"], axis=1)
    valid_dataset = valid_dataset.drop(["group", "id"], axis=1)
    test_dataset = test_dataset.drop(["group", "id"], axis=1)

    return train_dataset, valid_dataset, test_dataset


train_dataset, valid_dataset, test_dataset = prepare_data(data_lgbm)

train_X = train_dataset.drop(columns=["target_mro"], axis=1)

train_y = train_dataset["target_mro"]
train_pred = booster.predict(train_X)

# ----------------------------------------
# get best threshold from training set
best_threshold = 0.5
best_f1 = 0.0
thresholds = np.arange(0.02, 0.98, 0.01)

for threshold in thresholds:
    preds = (np.array(train_pred) > threshold).astype(int)
    f1 = f1_score(train_y, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"Best Threshold: {best_threshold:.2f}, Best F1 Score: {best_f1:.4f}")

# ----------------------------------------
# apply best threshold to test set
test_X = test_dataset.drop(columns=["target_mro"], axis=1)
test_y = test_dataset["target_mro"]

test_pred = booster.predict(test_X)
test_res = (np.array(test_pred) > best_threshold).astype(int)

test_f1 = f1_score(test_y, test_res)
test_accuracy = accuracy_score(test_y, test_res)
test_recall = recall_score(test_y, test_res)
test_precision = precision_score(test_y, test_res)

test_auc = roc_auc_score(test_y, test_pred)


results_df = pd.DataFrame(
    {
        "Trial Name": [trial_name],
        "Best Threshold": [best_threshold],
        "Best Test F1": [test_f1],
        "Best Test Accuracy": [test_accuracy],
        "Best Test Recall": [test_recall],
        "Best Test Precision": [test_precision],
        "Test AUC Score": [test_auc],
    }
)

Best Threshold: 0.61, Best F1 Score: 0.1908


In [9]:
results_df.to_csv(
    lgbm_different_year_compare_file_path,
    mode="a",
    index=False,
    header=not os.path.isfile(lgbm_different_year_compare_file_path),
)
